In [1]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display
import pickle

In [2]:
wlo = 0.5
whi = 2.0
hlo = -100.0
hhi = 10
vlo = 10.0**(hlo/20.0)

scilab_pass_ripple = 0.01
scilab_stop_ripple = 0.0005
rp0 = -20*np.log10(1-scilab_pass_ripple)
rs0 = -20*np.log10(scilab_stop_ripple)
try:
    with open("elliptic design.pkl","rb") as file:
        default_args = pickle.load(file)
except FileNotFoundError:
    default_args = {
        "Nstages":5,
        "rp":rp0,
        "rs":rs0 }
    
default_args

{'Nstages': 5,
 'rp': np.float64(0.08729610804900176),
 'rs': np.float64(66.02059991327963)}

In [3]:
def design_and_plot(Nstages=default_args["Nstages"],
                    rp=default_args["rp"],
                    rs=default_args["rs"]):
    Norder = Nstages*2
    b, a = signal.iirfilter(Norder, 1.0, rp=rp, rs=rs, 
                           btype='lowpass', analog=True,
                           ftype='ellip', output='ba')

    w, h = signal.freqs(b, a, np.logspace(np.log10(wlo),np.log10(whi),4000))
    fig = plt.figure("iFigure")
    ax = fig.add_subplot(1, 1, 1)
    ax.semilogx(w, 20 * np.log10(np.maximum(abs(h), vlo)))
    wst = 2.0**(1.0/12.0)
    ax.axvline(x=wst, color='green', linestyle='--', label='one semitone')
    ax.set_title('Elliptic lowpass prototype filter')
    ax.set_xlabel('Frequency [rad/s]')
    ax.set_ylabel('Amplitude [dB]')
    ax.axis((wlo, whi, hlo, hhi))
    ax.grid(which='both', axis='both')
    ax.legend()
    plt.show()
    return b, a

In [4]:
interactive_plot = interactive(design_and_plot,
                               Nstages=(1,6),
                               rp=(0.01,5.0,0.01),
                               rs=(10.0,90.0,2.0))

def write_header(button):
    args = interactive_plot.kwargs
    b, a = interactive_plot.result
    zeros, poles, k = signal.tf2zpk(b, a)
    with open("ellip_coeff.h","w") as f:
        f.write(f"#define N_STAGES {args['Nstages']}\n")
        f.write(f"static double ec_gain={k:.16e};\n")
        f.write("typedef struct {\n")
        f.write("\tdouble cden1;\n")
        f.write("\tdouble cden0;\n")
        f.write("\tdouble cnum0;\n")
        f.write("} ec_stage;\n")
        f.write("static ec_stage ec_stages[N_STAGES]={\n")
        for pole, zero in zip(poles[::2],zeros[::2]):
            cden1 = -2*pole.real
            cden0 = pole.real**2 + pole.imag**2
            cnum0 = zero.imag**2
            f.write(f"\t{{{cden1:.16e},{cden0:.16e},{cnum0:.16e}}},\n")
        f.write("};\n")
    with open("elliptic design.pkl", "wb") as file:
        pickle.dump(args, file)

write_button = widgets.Button(
    description='Write header file',
    disabled=False,
    button_style='',
    tooltip='Generate and write header file from current settings')

write_button.on_click(write_header)
display(write_button, interactive_plot)

Button(description='Write header file', style=ButtonStyle(), tooltip='Generate and write header file from curr…

interactive(children=(IntSlider(value=5, description='Nstages', max=6, min=1), FloatSlider(value=0.08729610804…